# Matriculas:
Rafael Lopes Murta (690213)
Caio Massote Andrade (725728)
Henrique Moura Souza Belo (732218)
Guilherme Vedovelo de Britto Claro (738097)

In [1]:
soma_matriculas = 690213 + 725728 + 732218 + 738097
N2 = soma_matriculas % 2
N3 = soma_matriculas % 3
print(f'matriculas mod 2 = {N2} = Compacidade \nmatriculas mod 3 = {N3} = EfficientNet')

matriculas mod 2 = 0 = Compacidade 
matriculas mod 3 = 1 = EfficientNet


## Imports

In [2]:
import cv2
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from os import makedirs
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from scipy.spatial.distance import mahalanobis
from torchvision import transforms

## Create Directories and load data

In [3]:
makedirs('train_test_images/train', exist_ok=True)
makedirs('train_test_images/test', exist_ok=True)
dir_list = ['scc', 'lsil', 'hsil', 'asc-h', 'asc-us', 'nfil']
for i in dir_list:
    makedirs(f'train_test_images/train/{i}', exist_ok=True)
    makedirs(f'train_test_images/test/{i}', exist_ok=True)

characteristics_df = pd.read_csv('characteristics.csv')

## Separate train and test images

In [4]:
train, test = train_test_split(characteristics_df, test_size=0.2, random_state=42, shuffle=True, stratify=characteristics_df['bethesda_system'])

## Save train and test images on their respective folders

In [5]:
def get_img_dir(bethesda_system, folder):
    if bethesda_system == 'SCC' or bethesda_system == 'LSIL' or bethesda_system == 'HSIL' or bethesda_system == 'ASC-H' or bethesda_system == 'ASC-US':
        return folder + '/' + bethesda_system.lower() + '/'
    elif bethesda_system == 'Negative for intraepithelial lesion':
        return folder + '/nfil/'

In [6]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

for index, row in train.iterrows():
    imagepath = f'{get_img_dir(row["bethesda_system"], "editted_images")}{row["cell_id"]}.jpg'
    image = Image.open(imagepath)
    new_imagepath = f'{get_img_dir(row["bethesda_system"], "train_test_images/train")}{row["cell_id"]}.jpg'
    image.save(new_imagepath)

for index, row in test.iterrows():
    imagepath = f'{get_img_dir(row["bethesda_system"], "editted_images")}{row["cell_id"]}.jpg'
    image = Image.open(imagepath)
    new_imagepath = f'{get_img_dir(row["bethesda_system"], "train_test_images/test")}{row["cell_id"]}.jpg'
    image.save(new_imagepath)

## Distancia de Mahalanobis

In [9]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')